In [3]:

import os
import numpy as np
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt

from brainglobe_heatmap import Heatmap


# Try to import the `save` function from brainglobe_utils.IO.image.
# If it’s not available, we'll fallback to Matplotlib's savefig.
try:
    from brainglobe_utils.IO.image import save as bg_save
    USE_BG_UTILS_SAVE = True
except ImportError:
    USE_BG_UTILS_SAVE = False

# ────────────────────────────────────────────────────────────────────────────────
# USER‐CONFIGURABLE PARAMETERS
# ────────────────────────────────────────────────────────────────────────────────

# 1) Path to the Excel file containing “region” and “value” columns
EXCEL_PATH = "fchange_heatmap_demonstrator.xlsx"

# 2) Name of the output folder for SVG slices
OUT_FOLDER = "coronal"  # will be created if it does not already exist

# 3) Number of coronal slices to generate
N_SLICES = 5

# 4) Coronal slab thickness (in micrometers)
THICKNESS = 100.0  # µm total slab thickness

# 5) Array of X‐positions (in µm) at which to sample coronal slices.
#    Here, we pick 10 positions evenly spaced from X=1000 µm to X=7000 µm.
SLICE_POSITIONS = np.linspace(1000.0, 10000.0, N_SLICES)

# 6) Fixed axis limits for all saved figures (in µm)
XLIM = (5500, -5500)
YLIM = (5000, -5000)

# ────────────────────────────────────────────────────────────────────────────────



# ────────────────────────────────────────────────────────────────────────────
# Step A: Load activation data from Excel and normalize to [0,1]
# ────────────────────────────────────────────────────────────────────────────
if not os.path.isfile(EXCEL_PATH):
    raise FileNotFoundError(f"Could not find '{EXCEL_PATH}' in the working directory.")

df = pd.read_excel(EXCEL_PATH, engine="openpyxl")
if "region" not in df.columns or "value" not in df.columns:
    raise KeyError("Excel file must contain columns named 'region' and 'value'.")

# Strip whitespace from region names (some Excel exports include trailing spaces)
df["region"] = df["region"].astype(str).str.strip()

vals = df["value"].astype(float)
vmin, vmax = vals.min(), vals.max()
#if vmax == vmin:
#    normalized = np.ones_like(vals) * 0.5
#else:
#    normalized = (vals - vmin) / (vmax - vmin)

region_values = dict(zip(df["region"], df["value"]))

# ────────────────────────────────────────────────────────────────────────────
# Step B: Create output directory if it doesn’t exist
# ────────────────────────────────────────────────────────────────────────────
os.makedirs(OUT_FOLDER, exist_ok=True)

# ────────────────────────────────────────────────────────────────────────────
# Step C: For each coronal slice position, build & save an SVG heatmap
# ────────────────────────────────────────────────────────────────────────────
for idx, xpos in enumerate(SLICE_POSITIONS):
    # 1) Instantiate the Heatmap for this coronal slice:
    heatmap = Heatmap(
        region_values,
        position=float(xpos),
        orientation="frontal",
        thickness=float(THICKNESS),
        vmin=0.0,
        vmax=np.ceil(vmax),
        cmap="hot",
        format="array"
    )

    # 2) Construct an output filename with zero-padded index (e.g. slice_00.svg):
    out_filename = os.path.join(OUT_FOLDER, f"slice_{idx:02d}.svg")
    print(f"[{idx+1}/{N_SLICES}] Generating coronal slice at X={xpos:.1f} µm → {out_filename}")

    # 3) Clear any existing figures
    plt.close("all")

    # 4) Draw the heatmap onto a new Figure
    fig = heatmap.plot(show=False)  # Many versions of Heatmap.plot() return a Matplotlib Figure

    # If Heatmap.plot() does not return a Figure but draws into current figure,
    # grab the current figure:
    if fig is None:
        fig = plt.gcf()

    # 5) Fix the axes limits on the current Axes
    ax = fig.axes[0] if fig.axes else plt.gca()
    ax.set_xlim(XLIM)
    ax.set_ylim(YLIM)

    # 6) Save as SVG, using brainglobe_utils if available
    if USE_BG_UTILS_SAVE:
        try:
            bg_save(fig, out_filename)
        except Exception as e:
            print(f"  → Warning: brainglobe_utils save failed ({e}); falling back to matplotlib savefig.")
            fig.savefig(out_filename, format="svg")
    else:
        fig.savefig(out_filename, format="svg")

    # 7) Close the figure to free memory
    plt.close(fig)

print("Done. All coronal‐slice SVGs saved to:", OUT_FOLDER)




[1/5] Generating coronal slice at X=1000.0 µm → coronal/slice_00.svg


<IPython.core.display.Javascript object>

<IPython.core.display.HTML object>

  → Warning: brainglobe_utils save failed ('module' object is not callable); falling back to matplotlib savefig.
[2/5] Generating coronal slice at X=3250.0 µm → coronal/slice_01.svg


<IPython.core.display.Javascript object>

...

<IPython.core.display.HTML object>

  → Warning: brainglobe_utils save failed ('module' object is not callable); falling back to matplotlib savefig.
[5/5] Generating coronal slice at X=10000.0 µm → coronal/slice_04.svg


<IPython.core.display.Javascript object>

<IPython.core.display.HTML object>

  → Warning: brainglobe_utils save failed ('module' object is not callable); falling back to matplotlib savefig.
Done. All coronal‐slice SVGs saved to: coronal


In [8]:
heatmap.scene.get_actors('ACA')

[brainrender.Actor: ACA-brain region]

In [43]:
from shapely import Polygon
from shapely.algorithms.polylabel import polylabel
from shapely.geometry.multipolygon import MultiPolygon

# find the correct position for labels
def find_annotation_position_inside_polygon(polygon_vertices):
    if polygon_vertices.shape[0] < 4:
        return None
    polygon = Polygon(polygon_vertices.tolist())

    if not polygon.is_valid:
        polygon = polygon.buffer(0)

    if polygon.geom_type == "MultiPolygon" and isinstance(
        polygon, MultiPolygon
    ):
        polygon = max(polygon.geoms, key=lambda p: p.area)

    label_position = polylabel(polygon, tolerance=0.1)
    return label_position.x, label_position.y

# find areas and coordinates for the current slice
projected, _= heatmap.slicer.get_structures_slice_coords(
            heatmap.regions_meshes, heatmap.scene.root
        )

segments = list()
for r, coords in projected.items():
    name, segment_nr = r.split("_segment_")
    x = coords[:, 0]
    y = coords[:, 1]
    # calculate area of polygon with Shoelace formula
    area = 0.5 * np.abs(
        np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1))
    )

    segments.append(
        {
            "name": name,
            "segment_nr": segment_nr,
            "coords": coords,
            "area": area,
        }
    )

segments.sort(key=lambda s: s["area"], reverse=True)

fig,ax = plt.subplots()
for segment in segments:
    name = segment["name"]

    segment_nr = segment["segment_nr"]
    coords = segment["coords"]

    ax.fill(
        coords[:, 0],
        coords[:, 1],
        color=heatmap.colors[name],
        label=name if segment_nr == "0" and name != "root" else None,
        lw=1,
        ec="k",
        zorder=-1 if name == "root" else None,
        alpha=0.3 if name == "root" else None,
    )


    if name != 'root':
        display_text = name
        annotation_pos = find_annotation_position_inside_polygon(coords)
        if annotation_pos is not None:
            ax.annotate(
                display_text,
                xy=annotation_pos,
                ha="center",
                va="center",
            )
ax.invert_yaxis()
ax.axis("equal")

<IPython.core.display.Javascript object>

<IPython.core.display.HTML object>

(-5064.032896009219, 5056.0352753752095, 3262.376689715628, -3787.7491085601473)

In [1]:
%matplotlib notebook

In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.colors as mcolors

from brainglobe_heatmap import Heatmap
from shapely import Polygon
from shapely.algorithms.polylabel import polylabel
from shapely.geometry import MultiPolygon
import plotly.graph_objs as go

from brainglobe_atlasapi import BrainGlobeAtlas  # for acronym → full name mapping

# ────────────────────────────────────────────────────────────────────────────────
# USER‐CONFIGURABLE PARAMETERS
# ────────────────────────────────────────────────────────────────────────────────

# 1) Path to the Excel file with “region” (acronym) and “value”
EXCEL_PATH = "fchange_heatmap_demonstrator.xlsx"

# 2) Number of coronal slices
N_SLICES = 30

# 3) Coronal slab thickness in µm
THICKNESS = 100.0

# 4) X‐positions in µm for coronal slices
SLICE_POSITIONS = np.linspace(1000.0, 10000.0, N_SLICES)

# 5) Matplotlib SVG limits (optional; not used here)
XLIM = (5500, -5500)
YLIM = (5500, -5500)

# 6) Fixed Plotly axis limits (Y and Z both from -5000 to +5000)
PLOTLY_RANGE = [-5000, 5000]

# 7) Plotly HTML output filename
PLOTLY_HTML = "slices_plotly.html"

# ────────────────────────────────────────────────────────────────────────────────
# HELPER FUNCTIONS
# ────────────────────────────────────────────────────────────────────────────────

def rgba_to_hex(color):
    """
    Convert a Matplotlib‐style color (name, hex, or RGBA tuple) into "#RRGGBB"
    """
    if isinstance(color, str):
        if color.startswith("#") and len(color) in (7, 9):
            return color[:7]
        try:
            rgba = mcolors.to_rgba(color)
            return mcolors.to_hex(rgba)
        except ValueError:
            pass
    if isinstance(color, (list, tuple)) and len(color) in (3, 4):
        return mcolors.to_hex(color)
    return "#cccccc"

def find_annotation_position_inside_polygon(polygon_vertices):
    """
    Given an (M × 2) NumPy array of [Y,Z] coords for a polygon,
    return (y, z) = a point inside via polylabel.
    """
    if polygon_vertices.shape[0] < 3:
        return None
    poly = Polygon(polygon_vertices.tolist())
    if not poly.is_valid:
        poly = poly.buffer(0)
    if isinstance(poly, MultiPolygon):
        poly = max(poly.geoms, key=lambda p: p.area)
    try:
        px, py = polylabel(poly, tolerance=0.1)
        return float(px), float(py)
    except Exception:
        return None

# ────────────────────────────────────────────────────────────────────────────────
# 1) LOAD ACTIVATION DATA
# ────────────────────────────────────────────────────────────────────────────────

if not os.path.isfile(EXCEL_PATH):
    raise FileNotFoundError(f"Could not find '{EXCEL_PATH}' in the working directory.")

df = pd.read_excel(EXCEL_PATH, engine="openpyxl")
if "region" not in df.columns or "value" not in df.columns:
    raise KeyError("Excel file must contain columns named 'region' and 'value'.")

df["region"] = df["region"].astype(str).str.strip()
region_values = dict(zip(df["region"], df["value"]))

# Determine global vmin/vmax for colorbar
vmin = 0.0
vmax = float(np.ceil(df["value"].max()))

# ────────────────────────────────────────────────────────────────────────────────
# 2) SET UP ATLAS FOR FULL NAMES
# ────────────────────────────────────────────────────────────────────────────────

atlas = BrainGlobeAtlas("allen_mouse_25um", check_latest=False)

# ────────────────────────────────────────────────────────────────────────────────
# 3) BUILD SLICES: COLLECT POLYGONS + COLORS
# ────────────────────────────────────────────────────────────────────────────────

segments_list = []      # segments_list[i] = list of dicts for slice i
common_color_map = {}   # region_name -> hex color

for idx, xpos in enumerate(SLICE_POSITIONS):
    print(f"[{idx+1}/{N_SLICES}] Generating slice at X = {xpos:.1f} µm")

    heatmap = Heatmap(
        region_values,
        position=float(xpos),
        orientation="frontal",
        thickness=float(THICKNESS),
        vmin=vmin,
        vmax=vmax,
        cmap="hot",
        format="array"
    )

    projected, _ = heatmap.slicer.get_structures_slice_coords(
        heatmap.regions_meshes, heatmap.scene.root
    )

    segments = []
    for r_key, coords in projected.items():
        if "_segment_" not in r_key:
            continue
        name, _ = r_key.split("_segment_")
        coords2d = coords[:, :2]   # [Y, Z]

        # Compute area via Shoelace formula
        x = coords2d[:, 0]
        y = coords2d[:, 1]
        area = 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

        segments.append({
            "name": name,
            "coords": coords2d,
            "area": area,
            "value": region_values.get(name, 0.0)
        })

    segments.sort(key=lambda s: s["area"], reverse=True)
    print(f"    → Found {len(segments)} polygons in this slice.")
    segments_list.append(segments)

    if not common_color_map:
        for reg, col in heatmap.colors.items():
            common_color_map[reg] = rgba_to_hex(col)

for i, segs in enumerate(segments_list):
    if len(segs) == 0:
        print(f"⚠️  Warning: Slice {i:02d} has 0 polygons (likely outside atlas).")

# ────────────────────────────────────────────────────────────────────────────────
# 4) BUILD PLOTLY FIGURE WITH SLIDER (NO FRAMES)
# ────────────────────────────────────────────────────────────────────────────────

all_traces = []         # flat list of Scatter traces for all slices
slice_index_map = []    # slice_index_map[k] = slice index for trace k

for i in range(N_SLICES):
    segs = segments_list[i]
    for seg in segs:
        name = seg["name"]
        coords = seg["coords"]           # (M, 2): coords[:,0]=Y, coords[:,1]=Z
        area = seg["area"]
        val = seg["value"]
        color_hex = common_color_map.get(name, "#cccccc")

        full_info = atlas.structures.get(name, {})
        full_name = full_info.get("name", "Unknown")

        hover_text = (
            f"{name}: {full_name}<br>"
            f"Area: {area:.1f} µm²<br>"
            f"Value: {val:.2f}"
        )

        trace = go.Scatter(
            x = coords[:, 0],
            y = coords[:, 1],
            fill = "toself",
            fillcolor = color_hex,
            line = dict(color="black", width=0.5),
            name = "",
            hoverinfo = "text",
            text = hover_text,
            visible = False
        )
        all_traces.append(trace)
        slice_index_map.append(i)

for k, sidx in enumerate(slice_index_map):
    if sidx == 0:
        all_traces[k].visible = True

steps = []
for i in range(N_SLICES):
    vis = [(sidx == i) for sidx in slice_index_map]
    step = {
        "method": "update",
        "label": str(i),
        "args": [
            {"visible": vis},
            {"title": f"Coronal Slice index: {i}"}
        ]
    }
    steps.append(step)

# Add colorbar trace
colorbar_trace = go.Scatter(
    x=[None],
    y=[None],
    mode="markers",
    marker=dict(
        colorscale="Hot",
        cmin=vmin,
        cmax=vmax,
        color=[vmax],
        showscale=True,
        colorbar=dict(
            title="Activation",
            thickness=15,
            len=0.75,
            yanchor="middle",
            y=0.5,
            tickmode="array",
            tickvals=[vmin, vmax],
            ticktext=[str(vmin), str(vmax)]
        )
    ),
    hoverinfo="none",
    showlegend=False
)
all_traces.insert(0, colorbar_trace)
slice_index_map.insert(0, -1)

fig = go.Figure(
    data = all_traces,
    layout = go.Layout(
        title = "Coronal Slice index: 0",
        xaxis = dict(
            title="Y (left → right) [µm]",
            range=PLOTLY_RANGE,
            zeroline=False,
            showgrid=False
        ),
        yaxis = dict(
            title="Z (dorsal → ventral) [µm]",
            range=[PLOTLY_RANGE[1], PLOTLY_RANGE[0]],
            zeroline=False,
            showgrid=False
        ),
        sliders = [dict(
            active = 0,
            pad = {"t": 50},
            steps = steps,
            currentvalue = {
                "prefix": "Slice index: ",
                "font": {"size": 14}
            }
        )],
        showlegend = False,
        paper_bgcolor = "white",
        plot_bgcolor = "white",
        margin = dict(l=50, r=50, t=80, b=50)
    )
)

fig.write_html(
    PLOTLY_HTML,
    include_plotlyjs="cdn",
    config={
        "displaylogo": False,
        "modeBarButtonsToRemove": [
            "toImage", "zoom2d", "pan2d", "select2d",
            "lasso2d", "zoomIn2d", "zoomOut2d", "autoScale2d",
            "resetScale2d", "hoverClosestCartesian", "hoverCompareCartesian"
        ]
    }
)
print(f"\nInteractive Plotly HTML (slider‐only) saved to: {PLOTLY_HTML}")


[1/30] Generating slice at X = 1000.0 µm
    → Found 3 polygons in this slice.
[2/30] Generating slice at X = 1310.3 µm
    → Found 3 polygons in this slice.
[3/30] Generating slice at X = 1620.7 µm
    → Found 5 polygons in this slice.
[4/30] Generating slice at X = 1931.0 µm
    → Found 11 polygons in this slice.
[5/30] Generating slice at X = 2241.4 µm
    → Found 10 polygons in this slice.
[6/30] Generating slice at X = 2551.7 µm
    → Found 11 polygons in this slice.
[7/30] Generating slice at X = 2862.1 µm
    → Found 15 polygons in this slice.
[8/30] Generating slice at X = 3172.4 µm
    → Found 15 polygons in this slice.
[9/30] Generating slice at X = 3482.8 µm
    → Found 16 polygons in this slice.
[10/30] Generating slice at X = 3793.1 µm
    → Found 16 polygons in this slice.
[11/30] Generating slice at X = 4103.4 µm
    → Found 16 polygons in this slice.
[12/30] Generating slice at X = 4413.8 µm
    → Found 14 polygons in this slice.
[13/30] Generating slice at X = 4724.1 µ

In [9]:
# TODO resolve te asymmerty of areas
# TODO starting from the middle frame